In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import re
import string
import gensim
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from nltk.stem.snowball import SnowballStemmer

In [ ]:
df = pd.read_csv('phishing_site_urls.csv')

In [ ]:
df.columns = [x.lower() for x in df.columns]

In [ ]:
df['Y'] = df['label'].apply(lambda x: 0 if x == 'good' else 1)

In [ ]:
def url_process(url):
    url = re.sub('[^a-zA-Z\ \n]', '.', url.lower())
    url =  re.sub('\.{1,}', ' ', url)
    url = url.split(' ')
    
    stemmer = SnowballStemmer("english")
    url = [stemmer.stem(word) for word in url]
    url = ' '.join(url)
    return url

In [ ]:
df['url_clean'] = df['url'].apply(url_process)

In [ ]:
%%time
cv=CountVectorizer(ngram_range=(1, 2))

X_train, X_test, y_train, y_test = train_test_split(df['url_clean'], df['Y'], test_size=0.2, random_state=5555)

cv_fit = cv.fit(X_train)
X_train = cv_fit.transform(X_train)

X_test = cv_fit.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC


from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report

lg = LogisticRegression(C=10,
                       penalty='l2',
                       solver='lbfgs')

nb = MultinomialNB(alpha= 0.1, class_prior= None, fit_prior=True)
svm = LinearSVC(C=1)


max_model = VotingClassifier(estimators=[('LogisticsRegression',lg),
                                        ('NultinomialNB',nb),
                                        ('SVM',svm)],
                                        voting='hard')
max_model.fit(X_train,y_train)
max_model.score(X_test,y_test)

In [ ]:
def new_input(url):
    a = url_process(url)
    a_trans = cv.transform(pd.Series(a))
    max_model_pred = max_model.predict(a_trans)
    print(a_trans)
    if max_model_pred ==1:
        return 'Warning! Site highly likely to be a phishing link! Proceed with caution!'
    else:
        return 'Great! Site is likely to be safe!'

In [ ]:
new_test = 'www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrcmd=_home-customer&nav=1/loading.php'

### Saving the ML model to pickle format

In [ ]:
import pickle
filename = 'g1-phishing-link-detection.sav'
pickle.dump(max_model, open(filename, 'wb'))

### Saving the Count Vectoriser for Training set to pickle format

In [ ]:
import pickle
pickle.dump(cv, open("vector.pickel", "wb"))

In [ ]:
cv = pickle.load(open("vector.pickel", "rb"))
new_input(new_test)